# Data Processing and splitting

In [1]:
from logic.processing import load_data, preproc, balance_dataset, data_filter
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay,classification_report
from sklearn.pipeline import make_pipeline
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras import layers,Sequential
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.callbacks import EarlyStopping

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Flotchi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/Flotchi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Flotchi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/Flotchi/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/Flotchi/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
2024-12-03 15:31:39.268561: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df_train = load_data('drugsComTrain_raw.csv')
df_test = load_data('drugsComTest_raw.csv')

In [3]:
df_train.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37


In [4]:
df_train_filter = data_filter(df_train)
df_test_filter = data_filter(df_test)

In [38]:
df_train_filter['review_length'].mean()

94.6271958419356

In [5]:
df_test_filter.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount,sentiment,review_length
0,163740,Mirtazapine,Depression,"""I&#039;ve tried a few antidepressants over th...",10,28-Feb-12,22,1,68
1,206473,Mesalamine,"Crohn's Disease, Maintenance","""My son has Crohn&#039;s disease and has done ...",8,17-May-09,17,1,48
3,39293,Contrave,Weight Loss,"""Contrave combines drugs that were used for al...",9,5-Mar-17,35,1,143
4,97768,Cyclafem 1 / 35,Birth Control,"""I have been on this birth control for one cyc...",9,22-Oct-15,4,1,149
7,169852,Amitriptyline,Migraine Prevention,"""This has been great for me. I&#039;ve been on...",9,21-Apr-09,32,1,64


In [6]:
df_train_prep = preproc(df_train_filter)
df_test_prep = preproc(df_test_filter)

In [7]:
X_train = df_train_prep['clean']
y_train = df_train_prep['sentiment']

In [8]:
X_test = df_test_prep['clean']
y_test = df_test_prep['sentiment']

In [9]:
X_train_b,y_train_b = balance_dataset(X_train,y_train)
X_test_b,y_test_b = balance_dataset(X_test,y_test)

In [10]:
len(y_train_b[y_train_b==0])==len(y_train_b[y_train_b==1])

True

# Tokenize

In [11]:
X_train_tk = [text_to_word_sequence(_) for _ in X_train_b]

In [12]:
X_test_tk = [text_to_word_sequence(_) for _ in X_test_b]

# Embedding

In [13]:
word2vec = Word2Vec(sentences=X_train_tk, vector_size=60, min_count=10, window=10)

In [14]:
def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])
        
    return np.array(embedded_sentence)

def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

In [15]:
X_train_embed = embedding(word2vec, X_train_tk)
X_test_embed = embedding(word2vec, X_test_tk)

# Padding

In [16]:
X_train_pad = pad_sequences(X_train_embed, dtype='float32', padding='post', maxlen=300)
X_test_pad = pad_sequences(X_test_embed, dtype='float32', padding='post', maxlen=300)

In [17]:
X_train_pad.shape,X_test_pad.shape

((56314, 300, 60), (19092, 300, 60))

In [18]:
word2vec.wv.vector_size

60

# Model Building

In [27]:
def init_model():
    
    model = Sequential()
    model.add(layers.Conv2D(16, (3,3), input_shape=(300, 60, 1), activation="relu"))
    model.add(layers.MaxPool2D(pool_size=(2,2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(50, activation='relu')) 
    model.add(layers.Dense(1,activation='sigmoid'))
    
    return model

In [28]:
model = init_model()

In [29]:
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

In [30]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 298, 58, 16)    │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 149, 29, 16)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 69136)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 50)             │     3,456,850 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,457,061 (13.19 MB)

 Trainable params: 3,457,061 (13.19 MB)

 Non-trainable params: 0 (0.00 B)

# Model training

In [31]:
es = EarlyStopping(patience=3, restore_best_weights=True)

model.fit(X_train_pad, y_train_b, 
          batch_size = 64,
          epochs=10,
          validation_split=0.3,
          callbacks=[es]
         )

Epoch 1/10
616/616 ━━━━━━━━━━━━━━━━━━━━ 199s 319ms/step - accuracy: 0.6270 - loss: 0.9322 - val_accuracy: 0.7351 - val_loss: 0.5247
Epoch 2/10
616/616 ━━━━━━━━━━━━━━━━━━━━ 196s 318ms/step - accuracy: 0.7701 - loss: 0.4856 - val_accuracy: 0.7826 - val_loss: 0.4829
Epoch 3/10
616/616 ━━━━━━━━━━━━━━━━━━━━ 205s 333ms/step - accuracy: 0.8047 - loss: 0.4337 - val_accuracy: 0.7793 - val_loss: 0.4733
Epoch 4/10
616/616 ━━━━━━━━━━━━━━━━━━━━ 186s 301ms/step - accuracy: 0.8272 - loss: 0.3927 - val_accuracy: 0.7897 - val_loss: 0.4668
Epoch 5/10
616/616 ━━━━━━━━━━━━━━━━━━━━ 187s 303ms/step - accuracy: 0.8505 - loss: 0.3431 - val_accuracy: 0.7664 - val_loss: 0.5155
Epoch 6/10
616/616 ━━━━━━━━━━━━━━━━━━━━ 177s 287ms/step - accuracy: 0.8629 - loss: 0.3177 - val_accuracy: 0.7859 - val_loss: 0.4825
Epoch 7/10
616/616 ━━━━━━━━━━━━━━━━━━━━ 185s 299ms/step - accuracy: 0.8851 - loss: 0.2710 - val_accuracy: 0.8035 - val_loss: 0.4749


# Evaluation

In [32]:
res = model.evaluate(X_test_pad, y_test_b, verbose=0)

In [33]:
print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

The accuracy evaluated on the test set is of 78.436%


In [ ]:
y_pred = model.predict(X_test_pad)

In [ ]:
y_pred = np.round(y_pred)

In [ ]:
print(classification_report(y_test_b, y_pred))